In [2]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt
import csv

# VERSION 2
start = (0, 0)
end = (0, 0)
pressed = False
COINS_SIZES = {
    0.01: 16.25,
    0.02: 18.75,
    0.10: 19.75,
    0.05: 21.25,
    0.20: 22.25,
    1.00: 23.25,
    0.50: 24.25,
    2.00: 25.75
}
sorted_coins = sorted(COINS_SIZES.items(), key=lambda x: x[1])

# Primer evento del ratón que toma captura de la foto
def mouse_events_camera(event, x, y, flags, params):
    global image, frame
    if event == cv2.EVENT_LBUTTONDOWN:
        image = frame.copy()

# Segundo evento del ratón que selecciona la moneda y calcula el tamaño de todas las monedas 
def mouse_events_picture(event, x, y, flags, params):
    global start, end, pressed
    if event == cv2.EVENT_LBUTTONDOWN: # Inicio del rectángulo
        start = (x, y)
        pressed = True
    elif event == cv2.EVENT_LBUTTONUP and pressed: # Fin del rectángulo
        end = (x, y)
        pressed = False
        
        # Dibuja el rectángulo final
        img_copy = image.copy()
        cv2.rectangle(img_copy, start, end, (255, 255, 255), 2)
        cv2.imshow("Frame", img_copy)

        # Procesa la moneda seleccionada
        coin = get_coin_size(image_original, 1, start, end)
        if not coin:
            print("No se ha podido detectar la moneda")
            return
        center, angle, diameter = coin        

        # Calcula todas las monedas de la imagen        
        show_sizes(image_original, center, angle, diameter)
    elif pressed:   # Mientras se arrastra el ratón
        end = (x, y)
        img_copy = image.copy()
        cv2.rectangle(img_copy, start, end, (255, 255, 255), -1)
        cv2.imshow("Frame", img_copy)

def get_contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Eliminar ruido
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.medianBlur(gray, 5)
    # Aplicar umbral OTSU para segmentar automáticamente
    _,img_th2 = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # Si hay más blanco que negro invertir
    cv2.imshow("Umbral OTSU", img_th2)
    if cv2.countNonZero(img_th2) < (img_th2.size / 2):
        img_th2 = cv2.bitwise_not(img_th2)
    # Aplicar un umbral fijo
    _, thresh = cv2.threshold(img_th2, 150, 255, cv2.THRESH_BINARY_INV)
    cv2.imshow("Umbral Fijo", thresh)
    # Encontrar contornos
    contornos, hierarchy = cv2.findContours(thresh,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    return contornos

def show_sizes(image, position, angle, reference_diameter):
    global text_color, thickness
    # Se considera que le diámetro de referencia pertenece a una moneda de un euro
    cantidad_total = 0
    contornos = get_contours(image)
    id = 0
    for c in contornos:
        cv2.drawContours(image, [c], -1, (255, 0, 0), 1)
        area = cv2.contourArea(c)
        if area > 10:
            # Conseguir el angulo de la elipse
            if c.shape[0] > 5:
                ellipse = cv2.fitEllipse(c)
                (cx, cy), (major_axis, minor_axis), angle_ellipse = ellipse
                
                center = (int(cx), int(cy))
                # La distancia solo importa en el eje vertical
                dist = cy - position[1]
                # Calcular la escala usando el angulo por parámetro
                alpha = 3.5 * (image.shape[0] - reference_diameter) / image.shape[0]
                scale = 1 - alpha * ((90-angle) / 90) * (dist / image.shape[0])
                # Ajustar la escala
                diameter = max(minor_axis, major_axis)
                diameter_scaled = diameter * scale
                # Calcular el tamaño de la moneda
                coin_size = COINS_SIZES[1] * diameter_scaled / reference_diameter
                coin_value = 2.00
                prev_value = 0
                # Buscar el valor de la moneda más cercana en tamaño
                for current_value, current_size in sorted_coins:
                    if coin_size < current_size:
                        if current_size == 16.25:
                            coin_value = 0.01
                        else:
                            prev_size = COINS_SIZES[prev_value] if prev_value else 0
                            distance_to_below = abs(coin_size - prev_size)
                            distance_to_upper = abs(current_size - coin_size)
                            if distance_to_below < distance_to_upper:
                                coin_value = prev_value
                            else:
                                coin_value = current_value
                        break
                    prev_value = current_value

                print(f"ID: {id} Moneda: {coin_value}€, Tamaño: {coin_size:.2f}mm, Escala: {scale:.2f}, Distancia: {dist}, diametro escalado: {diameter_scaled}, Diámetro.: {diameter}")
                # Dibuja los contornos de la figura
                cv2.ellipse(image, ellipse, (0, 255, 0), 2)
                # Dibuja el círculo de la moneda escalada
                cv2.circle(image, (int(cx), int(cy)), int(diameter_scaled/2), (255,0,0), 1)
                cv2.putText(image, f"ID: {id}", (center[0]-30, center[1]-15), 
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, thickness, cv2.LINE_8)
                cv2.putText(image, f"Add: {(diameter_scaled-diameter):.2f}", (center[0]-30, center[1]-5), 
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, thickness, cv2.LINE_8)
                cv2.putText(image, f"Si: {coin_size:.2f} Sc: {scale:.2}", (center[0]-30, center[1]+5), 
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, thickness, cv2.LINE_8)
                cv2.putText(image, f"V: {coin_value:.2f}", (center[0]-30, center[1]+15), 
                            cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, thickness, cv2.LINE_8)
                id += 1
                cantidad_total += coin_value
    cv2.putText(image, f"Total: {cantidad_total:.2f} euros", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, thickness, cv2.LINE_8)

def get_coin_size(image, amount, start, end):
    global dibujar_referencia
    # TODO: La idea es que con amount se pueda poner cualquier moneda, no solo 1 euro
    coin_image = image.copy()
    
    # Cortamos la imagen a la zona seleccionada
    new_start = (min(start[0], end[0]), min(start[1], end[1]))
    new_end = (max(start[0], end[0]), max(start[1], end[1]))
    coin_image = coin_image[new_start[1]:new_end[1], new_start[0]:new_end[0]]

    contours = get_contours(coin_image)  
    if contours:
        # Selecciona el contorno más grande (el exterior)
        # Ajustar una elipse al contorno (se necesitan >= 5 puntos)
        c_max = max(contours, key=cv2.contourArea)
        if len(c_max) >= 5:
            ellipse = cv2.fitEllipse(c_max)
            (cx, cy), (minor_axis, major_axis), angle = ellipse

            diameter = max(minor_axis, major_axis)

            # Se ajusta el centro a la posición global, no a la de la imagen recortada
            center = (int(cx) + new_start[0], int(cy) + new_start[1])

            # Calcula el ángulo corregido (a mi parecer)
            angle = abs(major_axis - minor_axis) / max(major_axis, minor_axis)
            angle = 90 - angle * 90
            
            # Dibuja círculos de referencia
            if dibujar_referencia:
                for i in range(30):
                    dist = 300 - i * 20
                    # Calcular la escala usando el angulo por parámetro
                    alpha = 3.5 * (image.shape[0] - diameter) / image.shape[0]
                    # alpha = 1
                    scale = 1 - alpha * ((90-angle) / 90) * (dist / image.shape[0])
                    diameter_scaled = diameter * scale
                    new_center = (int(center[0]), int(center[1] - dist))
                    print(f"Distancia: {dist}, Escala: {scale:.2f}, diametro escalado: {diameter_scaled}")
                    cv2.circle(image, new_center, int(diameter_scaled/2), (255,0,0), 1)

            print(f"Ángulo corregido: {angle:.2f} grados, Diámetro mayor: {major_axis:.2f}, Diámetro menor: {minor_axis:.2f} píxeles")
            print(f"Centro: {center}, Ángulo: {angle:.2f} grados, Diámetro mayor: {major_axis:.2f} píxeles")
            print(f"new_start: {new_start}, new_end: {new_end}, window size: {image_original.shape}")
            return center, angle, diameter
    else:
        print("No se han encontrado contornos")
        return None


In [5]:
# VERSION 2
vid = cv2.VideoCapture(0)

ret, frame = vid.read()
monedas = cv2.imread("Monedas.jpg")

image = None
cv2.imshow("Frame", frame)
cv2.setMouseCallback("Frame", mouse_events_camera)

text_color = (0,0,0)
thickness = 1
font_scale = 0.4
dibujar_referencia = True

while(True):      
    ret, frame = vid.read()
    frame = monedas.copy()

    frame_copy = frame.copy()
    cv2.putText(frame_copy, f"Click para congelar", (10,30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, thickness)
    cv2.imshow("Frame", frame_copy)

    if cv2.waitKey(20) == 27 or image is not None:
        # Libera el objeto de captura
        break

if image is None:
    print("No se ha congelado imagen")
    vid.release()
    cv2.destroyAllWindows()

image_original = image.copy()
cv2.setMouseCallback("Frame", mouse_events_picture)

while (True):
    image = image_original.copy()

    cv2.putText(image, f"Selecciona la moneda de 1 euro", (10,30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, thickness)
    cv2.imshow("Frame", image)

    if cv2.waitKey(20) == 27:
        break

# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Distancia: 300, Escala: 0.92, diametro escalado: 177.63679864351334
Distancia: 280, Escala: 0.93, diametro escalado: 178.62120290386767
Distancia: 260, Escala: 0.93, diametro escalado: 179.60560716422196
Distancia: 240, Escala: 0.94, diametro escalado: 180.5900114245763
Distancia: 220, Escala: 0.94, diametro escalado: 181.5744156849306
Distancia: 200, Escala: 0.95, diametro escalado: 182.55881994528494
Distancia: 180, Escala: 0.95, diametro escalado: 183.54322420563926
Distancia: 160, Escala: 0.96, diametro escalado: 184.52762846599356
Distancia: 140, Escala: 0.96, diametro escalado: 185.51203272634788
Distancia: 120, Escala: 0.97, diametro escalado: 186.4964369867022
Distancia: 100, Escala: 0.97, diametro escalado: 187.48084124705653
Distancia: 80, Escala: 0.98, diametro escalado: 188.46524550741086
Distancia: 60, Escala: 0.98, diametro escalado: 189.44964976776515
Distancia: 40, Escala: 0.99, diametro escalado: 190.4340540281195
Distancia: 20, Escala: 0.99, diametro escalado: 191.418